## Training progress

In [ ]:
import sys
import os
import pandas as pd

working_dir = '../output'
history_file = f'{working_dir}/history.csv'
if not os.path.exists(history_file):
    print(f'{history_file} not found')
    print('must train a model before running this')
    sys.exit()
    
df = pd.read_csv(history_file)

train_loss = []
val_loss = []
for _, group in df.groupby('epoch'):
    train_loss.append(group['train_loss'].mean())
    val_loss.append(group['epoch_val_loss'].mean())

loss_df = pd.DataFrame({'Training loss': train_loss, 'Validation loss': val_loss})
min_val_loss = df['epoch_val_loss'].min()
plots = loss_df.plot(xlabel='Epoch', grid=True, title=f'Minimum validation loss {min_val_loss:.4f}')

## Class activation maps on sample validation images

In [ ]:
import torch
from torch import nn

from util import get_class_names
from train import Trainer
from config import Config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dir = '../input'
model_file = f'{working_dir}/model.pth'
if not os.path.exists(model_file):
    print(f'{model_file} not found')
    sys.exit()
checkpoint = torch.load(model_file, map_location=device)
conf = Config(checkpoint['conf'])

trainer = Trainer(conf, input_dir, device, 1, checkpoint)
loader = trainer.val_loader
model = trainer.model

df = pd.read_csv(f'{input_dir}/train_metadata.csv', dtype=str)
class_names = get_class_names(df)

In [ ]:
import numpy as np
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from matplotlib import pyplot as plt

from util import search_layer

# search for the last convolution layer
target_layer = search_layer(model, torch.nn.Conv2d)
if not target_layer:
    print('Could not find conv layer')
    sys.exit()
print(f'Target layer for CAM: {target_layer}\n')

sigmoid = nn.Sigmoid()
images, labels = iter(loader).next()
model.eval()
with torch.no_grad():
    images = images.to(device)
    outputs = model(images)
    preds = sigmoid(outputs).round().cpu().numpy()
    num_rows = preds.shape[0]
    max_inds = outputs.argmax(axis=1).cpu().numpy()
    # this is to make sure that at least one class is predicted
    preds[range(num_rows), max_inds] = 1

cam = GradCAMPlusPlus(model, [target_layer], use_cuda=False)
cam_output = cam(input_tensor=images, targets=None)

# display class activation maps for this many images
count = min(8, len(images))
    
for idx in range(count):
    true_class_names = class_names[np.where(labels[idx])]
    pred_class_names = class_names[np.where(preds[idx])]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.subplots(1, 2)
    rgb_image = images[idx].cpu().numpy()
    # convert CHW to HWC
    rgb_image = rgb_image.transpose((1, 2, 0))
    rgb_image -= rgb_image.min()
    rgb_image /= rgb_image.max()

    im = show_cam_on_image(rgb_image, cam_output[idx], use_rgb=True)
    ax[0].set_title(f'Ground truth: {true_class_names}')
    ax[0].imshow(rgb_image)
    
    ax[1].set_title(f'Prediction: {pred_class_names}')
    ax[1].imshow(im)
plt.show()